# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [33]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

1. What test is appropriate for this problem? Does CLT apply?

Since we have 4870 samples *Z* test is approprate and CLT does apply because names were assigned randomly

In [6]:
# number of callbacks for black-sounding names
call_back_b = sum(data.call[(data.race == 'b')])
call_back_b

157.0

In [7]:
# number of callbacks for black-sounding names
call_back_w = sum(data.call[(data.race == 'w')])
call_back_w

235.0

In [8]:
total_call_back = sum(data.call)

In [9]:
call_back_rate_b = call_back_b / total_call_back
call_back_rate_b

0.40051020408163263

In [10]:
call_back_rate_w = call_back_w / total_call_back
call_back_rate_w

0.59948979591836737

 What are the null and alternate hypotheses?

In [30]:
# two-sample bootstrap hypothesis test: null hypothesis

# null hypothesis is that the means of black and white applicants is the same
# alternative hypothesis is that the means of not the same

# compute mean of all
mean_all = np.mean(data.call)
mean_all

0.08049281686544418

In [17]:
# arrays for black and white applicants
black = data.call[(data.race == 'b')]
white = data.call[(data.race == 'w')]

In [25]:
# Compute difference of mean
empirical_diff_means = np.mean(black) - np.mean(white)
empirical_diff_means

-0.03203285485506058

In [19]:
# generate shifted arrays
b_shifted = black - np.mean(black) + mean_all
w_shifted = white - np.mean(white) + mean_all

In [29]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates

In [23]:
# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_b = draw_bs_reps(b_shifted, np.mean, 10000)
bs_replicates_w = draw_bs_reps(w_shifted, np.mean, 10000)

In [42]:
# Get replicates of difference of means: bs_replicates
bs_replicates = bs_replicates_b - bs_replicates_w

In [28]:
# Compute and print p-value: p
p = np.sum(bs_replicates <= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)

p-value = 0.0


A small p-value indicates strong evidence against the null hypothesis, so we reject the null hypothesis that the means of black and white candidates call back mean is the same. A smaller p-value means that there is stronger evidence in favor of the alternative hypothesis.

In [38]:
# Compute 95% confidence intervals
conf_int_b = np.percentile(bs_replicates_b, [2.5,97.5])
conf_int_w = np.percentile(bs_replicates_w, [2.5,97.5])

In [39]:
conf_int_b

array([ 0.07104725,  0.0903491 ])

In [40]:
conf_int_w

array([ 0.06899384,  0.09240246])

In [45]:
np.mean(black)

0.0644763857126236

In [46]:
np.mean(white)

0.09650924056768417

Race does seem to have a statistically significant impact in terms of call back success.

However, it's hard to say that race is the most important factor in terms of call back success because we're not sure about the contents of the resume.  